In [ ]:
import numpy as np
import kaiwu as kw
import pandas as pd
kw.common.CheckpointManager.save_dir = "/tmp"
kw.license.init(user_id="xxxxxx", sdk_code="xxxxxxxx")

In [26]:
# Import distance matrix
w = np.array([[ 0, 13, 11, 16,  8],
              [13,  0,  7, 14,  9],
              [11,  7,  0, 10,  9],
              [16, 14, 10,  0, 12],
              [ 8,  9,  9, 12,  0]])

# Get the number of nodes
n = w.shape[0]

In [27]:
# Create qubo variable matrix
x = kw.qubo.ndarray((n, n), "x", kw.qubo.Binary)

# Get sets of edge and non-edge pairs
edges = [(u, v) for u in range(n) for v in range(n) if w[u, v] != 0]
no_edges = [(u, v) for u in range(n) for v in range(n) if w[u, v] == 0]

In [28]:
def is_edge_used(x, u, v):
    return kw.qubo.quicksum([x[u, j] * x[v, j + 1] for j in range(-1, n - 1)])

In [29]:
qubo_model = kw.qubo.QuboModel()
# TSP path cost
qubo_model.set_objective(kw.qubo.quicksum([w[u, v] * is_edge_used(x, u, v) for u, v in edges]))

# Node constraint: Each node must belong to exactly one position
qubo_model.add_constraint(x.sum(axis=0) == 1, "sequence_cons", penalty=20.0)

# Position constraint: Each position can have only one node
qubo_model.add_constraint(x.sum(axis=1) == 1, "node_cons", penalty=20.0)

# Edge constraint: Pairs without edges cannot appear in the path
qubo_model.add_constraint(kw.qubo.quicksum([is_edge_used(x, u, v) for u, v in no_edges]),
    "connect_cons", penalty=20.0)

In [30]:
# Perform calculation using SA optimizer
solver = kw.solver.SimpleSolver(kw.classical.SimulatedAnnealingOptimizer(initial_temperature=100,
                                                                         alpha=0.99,
                                                                         cutoff_temperature=0.001,
                                                                         iterations_per_t=10,
                                                                         size_limit=100))

sol_dict, qubo_val = solver.solve_qubo(qubo_model)
print(sol_dict)
print(qubo_val)


{'x[0][0]': np.float64(0.0), 'x[0][1]': np.float64(0.0), 'x[0][2]': np.float64(0.0), 'x[0][3]': np.float64(1.0), 'x[0][4]': np.float64(0.0), 'x[1][0]': np.float64(0.0), 'x[1][1]': np.float64(0.0), 'x[1][2]': np.float64(1.0), 'x[1][3]': np.float64(0.0), 'x[1][4]': np.float64(0.0), 'x[2][0]': np.float64(0.0), 'x[2][1]': np.float64(0.0), 'x[2][2]': np.float64(0.0), 'x[2][3]': np.float64(0.0), 'x[2][4]': np.float64(1.0), 'x[3][0]': np.float64(0.0), 'x[3][1]': np.float64(1.0), 'x[3][2]': np.float64(0.0), 'x[3][3]': np.float64(0.0), 'x[3][4]': np.float64(0.0), 'x[4][0]': np.float64(1.0), 'x[4][1]': np.float64(0.0), 'x[4][2]': np.float64(0.0), 'x[4][3]': np.float64(0.0), 'x[4][4]': np.float64(0.0)}
59.0
